In [1]:
import os
import numpy as np
from utils.visualisation import *
import sys
sys.path.append('../')
from helper.DataLoaderMNIST import DataLoader_MNIST
import torchvision.transforms as transforms
import torch
from resnet.model_def import ResNet
from VAE.model_def import VAE

In [2]:
max_seed=4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Get dataloader to plot
# Transformations for resnet
transformation_resnet = [transforms.ToTensor(),
                    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
                    ]
# Transdormations for vae
transformations_vae= [transforms.ToTensor(), 
                                # Normalize between -1 and 1
                                transforms.Normalize((0.5,), (0.5,)),
                                # Flatten the Image to a vector
                                transforms.Lambda(lambda x: x.view(-1) )
                                ]

# Load the data
batch_size=256
data_loader = DataLoader_MNIST(batch_size, transformation_resnet, transformation_resnet)
train_loader1, train_loader2 = data_loader.get_train_loader()
number_batches_to_plot=100

In [3]:
for seed1 in range(1,max_seed+1):
    path_model1 = f'../resnet/models/model_seed{seed1}.pth'
    #Load model1 and latent_space
    model1 = ResNet(pretrained=False).to(DEVICE)# Load pretrained weights for model2
    model1.load_state_dict(torch.load(path_model1,map_location = DEVICE))
    latent_space_model1,all_labels_model1 = get_latent_space_data(model1, train_loader1, DEVICE,number_batches_to_plot)
    #plotLatentTransformed(latent_space_model1,all_labels_model1, np.eye(latent_space_model1.shape[1]), pca_model1, f"Model 1 ResNet Seed {seed1}")

    for seed2 in range(seed1+1,max_seed+1):
        path_model2 = f'../resnet/models/model_seed{seed2}.pth'
        path_map = f'ResNet-LinearTransform/Linear_resnet_{seed1}_resnet_{seed2}_100.npy'

        # Load mapping
        A = np.load(path_map)
        # Load model2 and latent space
        model2 = ResNet(pretrained=False).to(DEVICE)# Load pretrained weights for model2
        model2.load_state_dict(torch.load(path_model2,map_location = DEVICE) )       
        latent_space_model2,all_labels_model2 = get_latent_space_data(model2, train_loader2, DEVICE,number_batches_to_plot)
        #pca_model2 = pca_def(latent_space_model2)
        #plotLatentTransformed(latent_space_model2,all_labels_model2, np.eye(latent_space_model2.shape[1]), pca_model2, f"Model 2 ResNet Seed {seed2}")
        #plotLatentTransformed(latent_space_model1,all_labels_model1, A, pca_model2, f"ResNet Seed {seed1} to {seed2} Transformed")
        #pca_model1 = pca_def(latent_space_model1)

        #Get average distance
        avg_distance = avg_transformed_distances(latent_space_model1, latent_space_model2, A)
        print(f"Seed {seed1} to Seed {seed2} average distance: {avg_distance}")

/home/hillary/miniconda3/envs/opti/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hillary/miniconda3/envs/opti/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


/home/hillary/miniconda3/envs/opti/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
